In [2]:
import numpy as np 
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification

random_state = 1
X, y = make_classification(n_samples=1500, n_features=3, n_informative=3, 
                    n_redundant=0, n_repeated=0, n_classes=2, n_clusters_per_class=2,
                    class_sep=1.5, flip_y=0,weights=[0.5,0.5], random_state=random_state)

In [3]:
!cat data/catalog.txt

data/dataset1.csv - root
data/dataset2.csv - root - Down
data/dataset3.csv - root - Down - Down, classes=[0 1], items<0>=17, items<1>=691, <couldn't go any further> LEAF accuracy=0.98
data/dataset4.csv - root - Down - Up
data/dataset5.csv - root - Down - Up - Down, classes=[0 1], items<0>=1, items<1>=3, <couldn't go any further> LEAF accuracy=0.75
data/dataset6.csv - root - Down - Up - Up, class=[0], items=7, rest=0,  <pure>  LEAF accuracy=1.00
data/dataset3.csv - root - Up, classes=[0 1], items<0>=725, items<1>=56, <couldn't go any further> LEAF accuracy=0.93


In [4]:
def readsub(name):
    data = np.genfromtxt(name, delimiter=',')
    data = np.array(data)
    py = data[:, data.shape[1] - 1]
    px = np.delete(data, data.shape[1] - 1, axis=1)
    return px, py
def localiza(X, px):
    enc = False
    for i in range(X.shape[0]):
        if all(X[i, :] == px):
            enc = True
            print(f" i={i} - X[{i}, :]={X[i, :]} - px={px} - y[{i}]={y[i]}")
    print("Encontrado:", enc)
    

In [5]:
px, py = readsub('data/dataset5.csv')
model = LinearSVC(random_state=1, max_iter=1000)
model.fit(px,py)
yp = model.predict(px)

In [15]:
print(yp)
print(py)

[1. 1. 1. 1.]
[1. 1. 0. 1.]


In [16]:
localiza(X, px[2, :])

i=1132 - X[1132, :]=[-0.41453617 -0.38206564  0.54849331] - px=[-0.41453617 -0.38206564  0.54849331] - y[1132]=0
Encontrado: True
